In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Load the dataset
data = pd.read_json('Dataset.json')

# Fix the 'label' issue
for i in range(0, len(data)):
    if data.annotation[i]['label'][0] == '1':
        data.annotation[i] = 1
    else:
        data.annotation[i] = 0

# Preprocessing
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['content'])

sequences = tokenizer.texts_to_sequences(data['content'])
padded = pad_sequences(sequences, maxlen=200, padding='post')

# Convert labels to categorical
labels = to_categorical(data['annotation'])

# Split data into training and testing sets
train_size = int(0.8 * len(padded))
train_padded, test_padded = padded[0:train_size], padded[train_size:]
train_labels, test_labels = labels[0:train_size], labels[train_size:]

# Define the model
model = Sequential()
model.add(Embedding(5000, 100, input_length=200))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

# Train the model
model.fit(train_padded, train_labels, epochs=30, batch_size=32, validation_data=(test_padded, test_labels), callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
loss, accuracy = model.evaluate(test_padded, test_labels)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

# Use the model for predictions
predictions = model.predict(test_padded)

Streaming output truncated to the last 5000 lines.
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-11-58df205904bf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-11-58df205904bf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-11-58df205904bf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

Epoch 1/30
500/500 [==============================] - 132s 258ms/step - loss: 0.6188 - accuracy: 0.6470 - val_loss: 0.3749 - val_accuracy: 0.8783
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


500/500 [==============================] - 122s 244ms/step - loss: 0.4298 - accuracy: 0.8054 - val_loss: 0.2310 - val_accuracy: 0.9235
Epoch 3/30
500/500 [==============================] - 123s 246ms/step - loss: 0.2872 - accuracy: 0.8823 - val_loss: 0.1850 - val_accuracy: 0.9313
Epoch 4/30
500/500 [==============================] - 122s 244ms/step - loss: 0.2095 - accuracy: 0.9156 - val_loss: 0.2216 - val_accuracy: 0.9133
Epoch 5/30
500/500 [==============================] - 122s 244ms/step - loss: 0.1594 - accuracy: 0.9367 - val_loss: 0.2047 - val_accuracy: 0.9308
Epoch 6/30
500/500 [==============================] - 122s 243ms/step - loss: 0.1359 - accuracy: 0.9463 - val_loss: 0.2067 - val_accuracy: 0.9313
Epoch 7/30
500/500 [==============================] - 118s 235ms/step - loss: 0.1131 - accuracy: 0.9551 - val_loss: 0.2590 - val_accuracy: 0.9165
Epoch 8/30
126/126 [==============================] - 9s 73ms/step - loss: 0.3253 - accuracy: 0.9018
Test loss: 0.32534608244895935, Te

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(test_padded)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(test_labels, axis=1)

print(classification_report(y_test, y_pred))

126/126 [==============================] - 7s 56ms/step
              precision    recall  f1-score   support

           0       1.00      0.90      0.95      4001
           1       0.00      0.00      0.00         0

    accuracy                           0.90      4001
   macro avg       0.50      0.45      0.47      4001
weighted avg       1.00      0.90      0.95      4001



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Real time

In [ ]:
pip install tweepy

In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
# Load the dataset
data = pd.read_json('Dataset.json')

# Fix the 'label' issue
for i in range(0, len(data)):
    if data.annotation[i]['label'][0] == '1':
        data.annotation[i] = 1
    else:
        data.annotation[i] = 0

Streaming output truncated to the last 5000 lines.
<ipython-input-19-990afc508e3c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-19-990afc508e3c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-19-990afc508e3c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-19-990afc508e3c>:9: SettingWithCopyWarning: 
A value is tryin

In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


data['label'] = data['annotation']
texts = data['content'].values
labels = data['label'].values
# Tokenize the texts
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post')

# Convert labels to categorical
labels = to_categorical(labels)

# Split data into training and testing sets
train_size = int(0.8 * len(padded_sequences))
train_padded, test_padded = padded_sequences[:train_size], padded_sequences[train_size:]
train_labels, test_labels = labels[:train_size], labels[train_size:]


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=200))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(64)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

# Train the model
model.fit(train_padded, train_labels, epochs=30, batch_size=32, validation_data=(test_padded, test_labels), callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
loss, accuracy = model.evaluate(test_padded, test_labels)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')


Epoch 1/30
500/500 [==============================] - 116s 220ms/step - loss: 0.6286 - accuracy: 0.6344 - val_loss: 0.3262 - val_accuracy: 0.9123
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


500/500 [==============================] - 107s 215ms/step - loss: 0.4558 - accuracy: 0.7903 - val_loss: 0.2267 - val_accuracy: 0.9228
Epoch 3/30
500/500 [==============================] - 108s 217ms/step - loss: 0.3187 - accuracy: 0.8667 - val_loss: 0.1862 - val_accuracy: 0.9278
Epoch 4/30
500/500 [==============================] - 108s 217ms/step - loss: 0.2328 - accuracy: 0.9060 - val_loss: 0.2764 - val_accuracy: 0.8878
Epoch 5/30
500/500 [==============================] - 107s 214ms/step - loss: 0.1718 - accuracy: 0.9357 - val_loss: 0.2304 - val_accuracy: 0.9210
Epoch 6/30
500/500 [==============================] - 108s 216ms/step - loss: 0.1404 - accuracy: 0.9459 - val_loss: 0.2806 - val_accuracy: 0.8998
Epoch 7/30
500/500 [==============================] - 108s 217ms/step - loss: 0.1158 - accuracy: 0.9553 - val_loss: 0.2392 - val_accuracy: 0.9175
Epoch 8/30
126/126 [==============================] - 6s 47ms/step - loss: 0.2435 - accuracy: 0.9250
Test loss: 0.2435002624988556, Tes